In [ ]:
import os
import numpy as np
import pandas as pd
from matplotlib import cm

In [ ]:
def set_pandas_display_options() -> None:
    """Set pandas display options."""
    # Ref: https://stackoverflow.com/a/52432757/
    display = pd.options.display

    display.max_columns = None
    display.max_rows = None
    display.max_colwidth = None
    display.width = None
    display.chop_threshold = None
    display.precision = None  # set as needed
    display.float_format = '{:,.8f}'.format

set_pandas_display_options()

# Define methods

## Latex

In [ ]:
def gap_bullet(a,b,c):
    return r'Gap {\Large\textcolor[rgb]{%f,%f,%f}{\ensuremath\bullet}}'%(a,b,c)

In [ ]:
def latex_float(f):
    """
        Based on https://stackoverflow.com/questions/13490292/format-number-using-latex-notation-in-python
    """
    float_str = f'{f:.1e}'
    if "e" in float_str:
        base, exponent = float_str.split("e")
        if int(exponent) > -10:
            return r"{0} \times 10^{{{1}\phantom{{0}}}}".format(base, int(exponent))
        else:
            return r"{0} \times 10^{{{1}}}".format(base, int(exponent))
    else:
        return float_str
    

def represent(x):

    integer_part = round(x)
    fractional_part = x - round(x)

    if integer_part >= 0:
        formatted_integer_part = "\phantom{-}" + str(integer_part)
    else:
        formatted_integer_part = str(integer_part)
    formatted_fractional_part = latex_float(fractional_part)

    if fractional_part>0:
        string_rep = "$" + formatted_integer_part + "{\color{gray} +" + formatted_fractional_part + "}$"
    elif fractional_part<0:
        string_rep = "$" + formatted_integer_part + "{\color{gray} " + formatted_fractional_part + "}$"
    else:
        string_rep = "$" + formatted_integer_part + "$"

    return(string_rep)

In [ ]:
x = 2.00000000124
print( represent(x) )

x = 1.999999999123
print( represent(x) )

x = 3
print( represent(x) )

## Chern coefficients

In [ ]:
def n_t1u1(theta,Ch_t1u1,Ch_t2u2,Ch_t1t2u1u2):
    return 1/2 * ( 1/(1-theta) * (Ch_t1u1+Ch_t2u2+2*theta*Ch_t1t2u1u2) + 1/(1+theta) * (Ch_t1u1-Ch_t2u2) )

def n_t2u2(theta,Ch_t1u1,Ch_t2u2,Ch_t1t2u1u2):
    return 1/2 * ( 1/(1-theta) * (Ch_t1u1+Ch_t2u2+2*theta*Ch_t1t2u1u2) - 1/(1+theta) * (Ch_t1u1-Ch_t2u2) )

def n_(theta,Ch_,Ch_t1u1,Ch_t2u2,Ch_t1t2u1u2):
    return Ch_ + ( theta**2 * Ch_t1t2u1u2 + theta * n_t1u1(theta,Ch_t1u1,Ch_t2u2,Ch_t1t2u1u2) + theta * n_t2u2(theta,Ch_t1u1,Ch_t2u2,Ch_t1t2u1u2) )

# Import Data

In [ ]:
files = [x[0] for x in os.walk('./data/')][1::]

df = pd.DataFrame()
for file in files:
    
    data = pd.read_json(file+'/params.json',orient='index').transpose()
    data["key"] = file[7:]
    
    if "tau1" in data:
        
        Chern_numbers = np.load(file+"/chern_numbers_tau1tau2u1u2.npy",allow_pickle=True)[()]

        for label in Chern_numbers:
            if label=="":
                data["IDS"]=Chern_numbers[label].real
            else:
                data[label]=Chern_numbers[label].real
        df = pd.concat([df,data])


display(df.set_index('key').sort_index(ascending=False))

# Process data

In [ ]:
keys = ['1700142847','1700143386','1700143909','1700144436','1700144967','1700145500']
thetas = np.zeros(len(keys))

ChernNumbers = np.zeros((8,len(keys)))

for i,key in enumerate(keys):
    thetas[i] = df.set_index('key').at[key,'q']
    ChernNumbers[0,i] = df.set_index('key').at[key,'IDS']
    ChernNumbers[1,i] = df.set_index('key').at[key,'tau1tau2']
    ChernNumbers[2,i] = df.set_index('key').at[key,'tau1u1']
    ChernNumbers[3,i] = df.set_index('key').at[key,'tau1u2']
    ChernNumbers[4,i] = df.set_index('key').at[key,'tau2u1']
    ChernNumbers[5,i] = df.set_index('key').at[key,'tau2u2']
    ChernNumbers[6,i] = df.set_index('key').at[key,'u1u2']
    ChernNumbers[7,i] = df.set_index('key').at[key,'tau1tau2u1u2']

ChernCoefficients = np.zeros((8,len(keys)))

ChernCoefficients[0] = n_(thetas,ChernNumbers[0],ChernNumbers[2],ChernNumbers[5],ChernNumbers[7])
ChernCoefficients[1] = ChernNumbers[1]
ChernCoefficients[2] = n_t1u1(thetas,ChernNumbers[2],ChernNumbers[5],ChernNumbers[7])
ChernCoefficients[3] = ChernNumbers[3]
ChernCoefficients[4] = ChernNumbers[4]
ChernCoefficients[5] = n_t2u2(thetas,ChernNumbers[2],ChernNumbers[5],ChernNumbers[7])
ChernCoefficients[6] = ChernNumbers[6]
ChernCoefficients[7] = ChernNumbers[7]

# Display data

In [ ]:
gaps_col = cm.Set1(range(len(keys)))

In [ ]:
CN = {r'$Ch_J$': [r'$Ch_\emptyset$',r'$Ch_{t_1t_2}$',r'$Ch_{t_1u_1}$','$Ch_{t_1u_2}$',r'$Ch_{t_2u_1}$',r'$Ch_{t_2u_2}$',r'$Ch_{u_1u_2}$',r'$Ch_{t_1t_2u_1u_2}$']}

for i in range(len(keys)):
    a,b,c,_ = gaps_col[i]
    CN[gap_bullet(a,b,c)] = ChernNumbers[:,i]

CN = pd.DataFrame(CN)

CC = {r'$n_J$' : [r'$n_\emptyset$',r'$n_{t_1t_2}$',r'$n_{t_1u_1}$',r'$n_{t_1u_2}$',r'$n_{t_2u_1}$',r'$n_{t_2u_2}$',r'$n_{u_1u_2}$',r'$n_{t_1t_2u_1u_2}$']}

for i in range(len(keys)):
    a,b,c,_ = gaps_col[i]
    CC[gap_bullet(a,b,c)] = ChernCoefficients[:,i]

CC = pd.DataFrame(CC)

display(CN,CC)

In [ ]:
print(CN.to_latex(index=False,
                  float_format="{:.8f}".format,
                  escape = False,
                  # column_format='l|llllll'
))

print(CC.to_latex(index=False,
                  float_format=represent,
                  escape = False,
                  # column_format='l|llllll'
))

with open('./tables/ChernNumber_Table.txt', 'w') as f:
    f.write(CN.to_latex(index=False,
                        float_format="{:.8f}".format,
                        escape = False,
                        # column_format='l|llllll'
                       )
           )
with open('./tables/ChernCoefficient_Table.txt', 'w') as f:
    f.write(CC.to_latex(index=False,
                        float_format=represent,
                        escape = False,
                        # column_format='l|llllll'
                       )
           )